In [2]:
import gurobipy as gp
import itertools as it

"""
implementation of solving profit maximization problem
author(s): 
yk796@cornell.edu
nd396@cornell.edu
"""

'\nimplementation of solving profit maximization problem\nauthor(s): \nyk796@cornell.edu\nnd396@cornell.edu\n'

In [3]:
# input
n_nodes = 2
n_alternative = 2

# O_demand = [1, 2, 2, 0]
# D_demand = [0, 3, 2, 0] 
O_demand = [3, 0]
D_demand = [0, 3] 

#TODO: discuss the difference between od pair and dealing with o, d demand separately
assert sum(O_demand) == sum(D_demand)

bpr_a = 2
bpr_b = 3

nodes = list(range(1, n_nodes+1))
alternatives = list(range(1, n_alternative+1))
arcs = list(it.permutations(nodes, 2))
#ods = list(it.permutations(nodes, 2))
ods = [(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]
T = {key:2 for key in list(it.product(ods, alternatives))}
ASC = {key:2 for key in list(it.product(ods, alternatives))}
D = {key:3 for key in ods}



In [19]:
def profit_maximization(n_nodes, n_alternative, O_demand, D_demand, T, ASC, bpr_a, bpr_b):
    eps = 0.001

    m = gp.Model()
    m.Params.DualReductions = 0 # to determine if the model is infeasible or unbounded
    #m.Params.OutputFlag = 0
    m.Params.NonConvex = 2
    m.Params.MIPGap = 0

    m._O = list(range(1, len(O_demand)+1))
    m._D = list(range(1, len(D_demand)+1))
    m._T = T
    m._ASC = ASC
    m._nodes = list(range(1, n_nodes+1))
    m._alternatives = list(range(1, n_alternative+1))
    m._arcs = list(it.permutations(m._nodes, 2)) 
    m._ods = [(id1+1, id2+1) for id1, o in enumerate(O_demand) for id2, d in enumerate(D_demand) if o>0 or d>0 if id1 != id2]

    m._theta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, ub=1, name='theta')
    m._y_vars = m.addVars(list(it.product(m._arcs, m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='y') # in fully connected graph, y=z
    m._z_vars = m.addVars(list(it.product(m._arcs, m._ods, m._alternatives)), vtype=gp.GRB.CONTINUOUS, lb=0, name='z') # in fully connected graph, y=z

    

    

    # create auxiliary variables to deal with non-linear objective function

    m._vvars = m.addVars(m._arcs, vtype=gp.GRB.CONTINUOUS, lb=0, name='v') 
    m._ln_theta_vars = m.addVars(list(it.product(m._ods, m._alternatives)), lb = -float('inf'), vtype=gp.GRB.CONTINUOUS, name='ln_theta')
    m._ind = m.addVars(m._ods, vtype=gp.GRB.BINARY, name="ind")
    m._profit_extracting = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, lb= 0, ub=1, name='extracting')
    m._profit_extracting_log = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, lb = -float('inf'), ub=0, name='extracting_log')
    #m._profit_extracting_term = m.addVars(m._ods, vtype=gp.GRB.CONTINUOUS, name='extracting_term')
    m._F = m.addVars(m._arcs, vtype=gp.GRB.CONTINUOUS, name='F')
    m._G = m.addVars(m._arcs, vtype=gp.GRB.CONTINUOUS, name='G')


    """add constraints"""
    # relationship between theta and z
    for j in m._alternatives:
        for (s, t) in m._ods:
            lhs = gp.quicksum(m._z_vars[(u, v), (s, t), j] for (u, v) in m._arcs if u == s)
            rhs = m._theta_vars[(s, t), j]
            m.addConstr(lhs == rhs, name = "constraint (7a)")

    for (s, t) in m._ods:
        lhs = gp.quicksum(m._theta_vars[(s, t), j] for j in m._alternatives)
        rhs = 1 #- eps
        m.addConstr(lhs <= rhs, name = "constraint (7b)")

    for j in m._alternatives:
        for a in m._arcs:
            for (s, t) in m._ods:
                m.addConstr(m._z_vars[a, (s, t), j] == m._y_vars[a, (s, t), j], name = "constraint (7c)") #TODO: modify this constraints


    # relationship to v_a
    for a in m._arcs:
        lhs = m._vvars[a]
        rhs = gp.quicksum(m._z_vars[a , (s, t), j] for (s, t) in m._ods for j in m._alternatives)
        m.addConstr(lhs == rhs, name = "equation (7d)")

    # flow conservation
    # for j in m._alternatives:
    #     for v in m._nodes:
    #         if (v != s) and (v != t):
    #             lhs = gp.quicksum(m._y_vars[(u, v) , (s, t), j] for u in m._nodes if u != v for (s, t) in m._ods)
    #             rhs = gp.quicksum(m._y_vars[(v, u) , (s, t), j] for u in m._nodes if u != v for (s, t) in m._ods)
    #             m.addConstr(lhs == rhs, name = "constraint (7e)")

    # objective function
    for j in m._alternatives:
        for (s, t) in m._ods:
            m.addGenConstrLog(m._theta_vars[(s, t), j], m._ln_theta_vars[(s, t), j], name = "ln_theta") 

    for (s, t) in m._ods:
        m.addConstr(m._profit_extracting[s,t] == 1 - gp.quicksum(m._theta_vars[(s, t), j] for j in m._alternatives), name ='extract')
        m.addGenConstrLog(m._profit_extracting[s,t], m._profit_extracting_log[s,t], name = "ln_profit")

    


        # M = 100 # large M is set to total demand
        # m.addConstr(m._profit_extracting[s,t] >= 0 + eps - M * (1 - m._ind[s,t]), name="bigM_constr1") # for strict inequality, we need eps
        # m.addConstr(m._profit_extracting[s,t] <= 0 + M * m._ind[s,t], name="bigM_constr2")
        # # Add indicator constraints to deal with non zero g variables
        # m.addConstr((m._ind[s,t] == 1) >> (m._profit_extracting_term[s,t] == m._profit_extracting_log[s,t]), name="gvar_nonzero")
        # m.addConstr((m._ind[s,t] == 0) >> (m._profit_extracting_term[s,t] == m._profit_extracting[s,t]), name="gvar_zero")


    # TODO: include general bpr function instead of the current linear function
    # Define F and G
    for a in m._arcs:
        lhs = m._F[a]
        rhs = 0.5 * bpr_a * m._vvars[a] * m._vvars[a] + bpr_b * m._vvars[a]
        m.addConstr(lhs == rhs, name = "F_function")

        lhs = m._G[a]
        rhs = 0.5 * bpr_a * m._vvars[a] * m._vvars[a] + bpr_b * m._vvars[a]
        m.addConstr(lhs == rhs, name = "G_function")
        

    # define objective function
    m.setObjective(gp.quicksum(D[s,t]* gp.quicksum(m._theta_vars[(s, t), j] * (m._T[(s, t), j]-m._ASC[(s, t), j]) for j in m._alternatives) for (s, t) in m._ods) # objective function (a)
                   + gp.quicksum(D[s,t] * gp.quicksum(m._theta_vars[(s, t), j] * m._ln_theta_vars[(s, t), j] for j in m._alternatives) for (s, t) in m._ods) #objective function (b)
                   - gp.quicksum(D[s,t] * m._profit_extracting_log[s,t] for (s,t) in m._ods) #objective function (c)
                   )


    m.update()
    m.optimize()

    # m.computeIIS() # this helps us to identify constraints that are responsible to make the model infeasible.
    # m.write("model.ilp")

    for v in m.getVars():
        print(f"{v.VarName} = {v.X}")



    if m.Status == 3:
        return None, None
    else:
        return m._theta_vars, m.ObjVal


theta, obj = profit_maximization(n_nodes, n_alternative, O_demand, D_demand, T, ASC, bpr_a, bpr_b)
print(theta, obj)

Set parameter DualReductions to value 0
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 21 columns and 23 nonzeros
Model fingerprint: 0xecce0ace
Model has 2 quadratic objective terms
Model has 4 quadratic constraints
Model has 3 general constraints
Variable types: 20 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [3e+00, 3e+00]
  QObjective range [6e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve added 55 rows and 1323 columns
Presolve time: 0.03s
Presolved: 76 rows, 1348 columns, 24396 nonzeros
Presolved model has 4 bilinear constraint(s)
Variable types: 1320 continuous, 28 integer (4 binary)

Root relaxation: